In [ ]:
# Optimized Solar Efficiency Prediction System
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OrdinalEncoder, RobustScaler
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# Install and import advanced libraries
try:
    from lightgbm import LGBMRegressor
    HAS_LIGHTGBM = True
    print("✓ LightGBM available")
except ImportError:
    print("Installing LightGBM...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "lightgbm"])
    from lightgbm import LGBMRegressor
    HAS_LIGHTGBM = True

try:
    from xgboost import XGBRegressor
    HAS_XGBOOST = True
    print("✓ XGBoost available")
except ImportError:
    print("Installing XGBoost...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "xgboost"])
    from xgboost import XGBRegressor
    HAS_XGBOOST = True

try:
    from catboost import CatBoostRegressor
    HAS_CATBOOST = True
    print("✓ CatBoost available")
except ImportError:
    print("Installing CatBoost...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "catboost"])
    from catboost import CatBoostRegressor
    HAS_CATBOOST = True

print("\n=== LOADING DATA ===")

# Load data with correct file names
def load_data():
    """Load train and test data with multiple path attempts"""
    paths_to_try = [
        ('train_cleaned.csv', 'test_cleaned.csv'),
        ('./train_cleaned.csv', './test_cleaned.csv'),
        ('/content/train_cleaned.csv', '/content/test_cleaned.csv'),
        ('/content/drive/MyDrive/train_cleaned.csv', '/content/drive/MyDrive/test_cleaned.csv')
    ]

    for train_path, test_path in paths_to_try:
        try:
            train_data = pd.read_csv(train_path)
            test_data = pd.read_csv(test_path)
            print(f"✓ Data loaded from: {train_path}")
            print(f"  Train shape: {train_data.shape}")
            print(f"  Test shape: {test_data.shape}")
            return train_data, test_data
        except FileNotFoundError:
            continue

    raise FileNotFoundError("Could not find train_cleaned.csv and test_cleaned.csv files")

train, test = load_data()

# Store test IDs
test_ids = test['id'].copy()

print(f"Training columns: {list(train.columns)}")
print(f"Test columns: {list(test.columns)}")

# Check target variable
if 'efficiency' in train.columns:
    print(f"Target variable statistics:")
    print(train['efficiency'].describe())
else:
    print("Warning: 'efficiency' column not found!")

print("\n=== DATA PREPROCESSING ===")

def clean_data(df):
    """Clean and preprocess data"""
    df = df.copy()

    # Convert numeric columns
    numeric_cols = ['humidity', 'wind_speed', 'pressure', 'voltage', 'current',
                   'module_temperature', 'temperature']

    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
            # Fill missing values with median
            df[col].fillna(df[col].median(), inplace=True)

            # Remove extreme outliers
            Q1 = df[col].quantile(0.05)
            Q3 = df[col].quantile(0.95)
            df[col] = np.clip(df[col], Q1, Q3)

    # Handle categorical columns
    categorical_cols = ['string_id', 'error_code', 'installation_type']
    for col in categorical_cols:
        if col in df.columns:
            df[col].fillna('unknown', inplace=True)

    return df

# Clean data
train = clean_data(train)
test = clean_data(test)

# Encode categorical variables - using already encoded data
# The data appears to be pre-encoded with binary columns for categorical features
print("✓ Using pre-encoded categorical features")

print("\n=== FEATURE ENGINEERING ===")

def create_features(df):
    """Create optimized feature set"""
    df = df.copy()

    # Core electrical features
    df['power'] = df['voltage'] * df['current']
    df['resistance'] = df['voltage'] / (df['current'] + 1e-8)

    # Temperature features
    df['temp_diff'] = df['module_temperature'] - df['temperature']
    df['temp_ratio'] = df['module_temperature'] / (df['temperature'] + 273.15)

    # Solar efficiency features
    df['irradiance_temp'] = df['irradiance'] * df['temperature']
    df['irradiance_power'] = df['irradiance'] * df['power']
    df['soiling_effect'] = df['soiling_ratio'] * df['irradiance']

    # Environmental interactions
    df['humidity_temp'] = df['humidity'] * df['temperature']
    df['pressure_temp'] = df['pressure'] * df['temperature']
    df['wind_temp'] = df['wind_speed'] * df['temperature']
    df['cloud_irradiance'] = df['cloud_coverage'] * df['irradiance']

    # Panel condition features
    df['age_maintenance'] = df['panel_age'] * df['maintenance_count']
    df['age_soiling'] = df['panel_age'] * df['soiling_ratio']

    # Power efficiency indicators
    df['power_per_temp'] = df['power'] / (df['temperature'] + 273.15)
    df['power_per_irradiance'] = df['power'] / (df['irradiance'] + 1e-8)
    df['voltage_efficiency'] = df['voltage'] / (df['module_temperature'] + 273.15)

    # Polynomial features for key variables
    df['voltage_squared'] = df['voltage'] ** 2
    df['current_squared'] = df['current'] ** 2
    df['power_squared'] = df['power'] ** 2
    df['irradiance_squared'] = df['irradiance'] ** 2

    # Log transformations
    df['power_log'] = np.log1p(df['power'])
    df['voltage_log'] = np.log1p(df['voltage'])
    df['irradiance_log'] = np.log1p(df['irradiance'])

    return df

# Apply feature engineering
train = create_features(train)
test = create_features(test)

# Prepare features
X = train.drop(columns=['id', 'efficiency'])
y = train['efficiency']
X_test = test.drop(columns=['id'] + (['efficiency'] if 'efficiency' in test.columns else []))

# Feature scaling
scaler = RobustScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

print(f"Final feature count: {X.shape[1]}")

print("\n=== MODEL TRAINING ===")

# Optimized model configurations
models = {
    'LightGBM': LGBMRegressor(
        objective='regression',
        n_estimators=1000,
        learning_rate=0.05,
        max_depth=8,
        num_leaves=63,
        feature_fraction=0.8,
        bagging_fraction=0.8,
        bagging_freq=5,
        reg_alpha=0.1,
        reg_lambda=0.1,
        random_state=42,
        verbosity=-1
    ),

    'XGBoost': XGBRegressor(
        objective='reg:squarederror',
        n_estimators=1000,
        learning_rate=0.05,
        max_depth=8,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.1,
        reg_lambda=0.1,
        random_state=42,
        verbosity=0
    ),

    'CatBoost': CatBoostRegressor(
        iterations=1000,
        learning_rate=0.05,
        depth=8,
        l2_leaf_reg=3,
        random_seed=42,
        verbose=False
    ),

    'RandomForest': RandomForestRegressor(
        n_estimators=300,
        max_depth=15,
        min_samples_split=5,
        min_samples_leaf=2,
        random_state=42,
        n_jobs=-1
    )
}

# Cross-validation setup
kf = KFold(n_splits=5, shuffle=True, random_state=42)
n_models = len(models)

# Initialize prediction arrays
oof_preds = np.zeros((len(X), n_models))
test_preds = np.zeros((len(X_test), n_models))
model_scores = []

print("Training models with 5-fold CV...")

for fold, (train_idx, val_idx) in enumerate(kf.split(X_scaled, y)):
    print(f"\nFold {fold + 1}/5")

    X_train, X_val = X_scaled.iloc[train_idx], X_scaled.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    fold_scores = []

    for i, (name, model) in enumerate(models.items()):
        # Train model with simplified approach
        try:
            if name == 'LightGBM':
                model.fit(X_train, y_train,
                         eval_set=[(X_val, y_val)])
            elif name == 'XGBoost':
                model.fit(X_train, y_train,
                         eval_set=[(X_val, y_val)],
                         verbose=False)
            elif name == 'CatBoost':
                model.fit(X_train, y_train,
                         eval_set=(X_val, y_val))
            else:
                model.fit(X_train, y_train)
        except Exception as e:
            print(f"  Warning: {name} training issue, using basic fit")
            model.fit(X_train, y_train)

        # Predictions
        val_pred = model.predict(X_val)
        test_pred = model.predict(X_test_scaled)

        # Store predictions
        oof_preds[val_idx, i] = val_pred
        test_preds[:, i] += test_pred / kf.n_splits

        # Score
        rmse = mean_squared_error(y_val, val_pred) ** 0.5
        fold_scores.append(rmse)
        print(f"  {name:12s} RMSE: {rmse:.6f}")

    model_scores.append(fold_scores)

print("\n=== MODEL PERFORMANCE ===")
avg_scores = np.mean(model_scores, axis=0)
model_names = list(models.keys())

for i, name in enumerate(model_names):
    oof_rmse = mean_squared_error(y, oof_preds[:, i]) ** 0.5
    oof_r2 = r2_score(y, oof_preds[:, i])
    print(f"{name:12s} - CV RMSE: {avg_scores[i]:.6f}, OOF RMSE: {oof_rmse:.6f}, R²: {oof_r2:.4f}")

print("\n=== ENSEMBLE PREDICTION ===")

# Create meta-features for stacking
meta_train = oof_preds.copy()
meta_test = test_preds.copy()

# Add ensemble features
meta_train_enhanced = np.column_stack([
    meta_train,
    np.mean(meta_train, axis=1),  # Average
    np.std(meta_train, axis=1),   # Standard deviation
    np.max(meta_train, axis=1),   # Maximum
    np.min(meta_train, axis=1)    # Minimum
])

meta_test_enhanced = np.column_stack([
    meta_test,
    np.mean(meta_test, axis=1),
    np.std(meta_test, axis=1),
    np.max(meta_test, axis=1),
    np.min(meta_test, axis=1)
])

# Scale meta features
meta_scaler = RobustScaler()
meta_train_scaled = meta_scaler.fit_transform(meta_train_enhanced)
meta_test_scaled = meta_scaler.transform(meta_test_enhanced)

# Final ensemble model
ensemble_model = Ridge(alpha=1.0)
ensemble_model.fit(meta_train_scaled, y)

# Final predictions
final_predictions = ensemble_model.predict(meta_test_scaled)

# Ensemble performance
ensemble_oof = ensemble_model.predict(meta_train_scaled)
ensemble_rmse = mean_squared_error(y, ensemble_oof) ** 0.5
ensemble_r2 = r2_score(y, ensemble_oof)

print(f"Ensemble RMSE: {ensemble_rmse:.6f}")
print(f"Ensemble R²: {ensemble_r2:.4f}")

# Best single model for comparison
best_single_idx = np.argmin(avg_scores)
best_single_rmse = mean_squared_error(y, oof_preds[:, best_single_idx]) ** 0.5

improvement = ((best_single_rmse - ensemble_rmse) / best_single_rmse) * 100
print(f"Improvement over best single model: {improvement:.2f}%")

print("\n=== FINAL SUBMISSION ===")

# Ensure predictions are within reasonable bounds
final_predictions = np.clip(final_predictions, 0, 1)

# Create submission
submission = pd.DataFrame({
    'id': test_ids,
    'efficiency': final_predictions
})

print(f"Submission shape: {submission.shape}")
print(f"Prediction range: [{final_predictions.min():.4f}, {final_predictions.max():.4f}]")
print(f"Prediction mean: {final_predictions.mean():.4f}")

# Save submission
submission.to_csv('solar_efficiency_submission.csv', index=False)
print("✓ Submission saved as 'solar_efficiency_submission.csv'")

print("\nSample predictions:")
print(submission.head())

print("\n" + "="*50)
print("🎯 OPTIMIZATION COMPLETE")
print("="*50)
print(f"✓ Final Ensemble RMSE: {ensemble_rmse:.6f}")
print(f"✓ Final Ensemble R²: {ensemble_r2:.4f}")
print(f"✓ Features Used: {X.shape[1]}")
print(f"✓ Models Trained: {len(models)}")
print(f"✓ Predictions Generated: {len(submission)}")
print("="*50)

✓ LightGBM available
✓ XGBoost available
✓ CatBoost available

=== LOADING DATA ===
✓ Data loaded from: train_cleaned.csv
  Train shape: (20000, 24)
  Test shape: (12000, 24)
Training columns: ['id', 'temperature', 'irradiance', 'humidity', 'panel_age', 'maintenance_count', 'soiling_ratio', 'voltage', 'current', 'module_temperature', 'cloud_coverage', 'wind_speed', 'pressure', 'efficiency', 'string_id_A1', 'string_id_B2', 'string_id_C3', 'string_id_D4', 'error_code_E00', 'error_code_E01', 'error_code_E02', 'installation_type_dual-axis', 'installation_type_fixed', 'installation_type_tracking']
Test columns: ['id', 'temperature', 'irradiance', 'humidity', 'panel_age', 'maintenance_count', 'soiling_ratio', 'voltage', 'current', 'module_temperature', 'cloud_coverage', 'wind_speed', 'pressure', 'efficiency', 'string_id_A1', 'string_id_B2', 'string_id_C3', 'string_id_D4', 'error_code_E00', 'error_code_E01', 'error_code_E02', 'installation_type_dual-axis', 'installation_type_fixed', 'installa